In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import os, shutil, pathlib
from PIL import Image

In [ ]:
def make_graphs(obj):
    epochs = len(obj.history['loss'])
    fig, axes = plt.subplots(2, 1, figsize=(10, 8)) 
    make_graphs_loss(obj, epochs, axes[0]) 
    make_graphs_accuracy(obj, epochs, axes[1])  
    plt.tight_layout()

def make_graphs_loss(obj, epochs, ax):
    # makes graph from object and epochs
    eps = [i for i in range(epochs)]
    train_acc = obj.history['loss']
    val_acc = obj.history['val_loss']
    # Now make plot
    sns.lineplot(x=eps[1:], y=train_acc[1:], color='b', label='Training Loss', ax=ax)
    sns.lineplot(x=eps[1:], y=val_acc[1:], color='r', label='Validation Loss', ax=ax)
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.set_title('Training vs Validation')
    ax.legend()

def make_graphs_accuracy(obj, epochs, ax):
    # makes graph from object and epochs
    eps = [i for i in range(epochs)]
    train_acc = obj.history['accuracy']
    val_acc = obj.history['val_accuracy']
    # Now make plot
    sns.lineplot(x=eps[1:], y=train_acc[1:], color='b', label='Training Accuracy', ax=ax)
    sns.lineplot(x=eps[1:], y=val_acc[1:], color='r', label='Validation Accuracy', ax=ax)
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    ax.set_title('Training vs Validation')
    ax.legend()

In [ ]:
train_dataset = image_dataset_from_directory(
    r'images_sorted_tvts/train',
    image_size=(255, 255),
    batch_size=32,
    labels='inferred',
    label_mode='categorical')

val_data = image_dataset_from_directory(r'images_sorted_tvts/validation',
    image_size=(255, 255),
    batch_size=32,
    labels='inferred',
    label_mode='categorical')

In [ ]:
VGG16_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
VGG16_base.trainable = False

inputs = keras.Input(shape=(255, 255, 3))
x = inputs
x = keras.applications.vgg16.preprocess_input(x)  # scale the data
x = VGG16_base(x)
x = layers.Flatten()(x)
x = layers.Dense(64)(x)   # only the weights in the dense layers will be trained
outputs = layers.Dense(37, activation="softmax")(x)     # only the weights in the dense layers will be trained
VGG16_Model = keras.Model(inputs, outputs)
VGG16_Model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

VGG16_hist = VGG16_Model.fit(train_dataset, epochs=5, validation_data= val_data)

In [ ]:
make_graphs(VGG16_hist)

In [ ]:
# Xception Model
Xception_base = keras.applications.Xception(
    weights="imagenet",
    include_top=False)
Xception_base.trainable = False

inputs = keras.Input(shape=(255, 255, 3))
x = inputs
x = keras.applications.xception.preprocess_input(x)
x = Xception_base(x)
x = layers.Flatten()(x)
x = layers.Dense(64)(x)
outputs = layers.Dense(37, activation="softmax")(x)
Xception_Model = keras.Model(inputs, outputs)
Xception_Model.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"])

Xception_hist = Xception_Model.fit(train_dataset, epochs=5, validation_data= val_data)

In [ ]:
make_graphs(Xception_hist)